# 微分定义
微分是指函数在某一点的线性近似，表示函数值的微小变换量。对于函数 $y = f(x)$， 微分 $\mathrm{d} y$ 可以表示为

$$\mathrm{d} y = f'(x) \mathrm{d} x$$

导数是指函数在某一点的变化率，为微分的系数。可以通过手动计算、符号位分、数值微分、自动微分计算导数或梯度
## 符号微分
通过代数规则对表达式解析求导，可以得到精确的导数表达式，但表达式膨胀，难以处理分支、循环等程序结构
## 数值微分
用有限差分近似导数：$f'(x) \approx \frac{f(x+h) - f(x)}{h}$ 实现简单，但存在截断误差和舍入误差
## 自动微分
将函数分解为基本操作，利用链式法则计算导数（前向/反向模式），精确高效，支持复杂程序结构，但实现复杂，需要跟踪计算过程

### 前向模式
* 原理：按照计算顺序，逐层计算中间变量及其导数。
* 使用场景：输入维度低，输出维度高
* 过程：
  * 每个中间变量 $v_i$ 同时计算其值 $v_i$ 和导数 $\dot{v}_i = \frac{\mathrm{d}v_i}{\mathrm{d}x}$
  * 通过链式法则逐步累积导数
* 示例：
  计算 $f(x, y) = x \cdot y + sin(x)$ 在点(x = 2, y = 3) 处对 $x$ 的导数：

In [ ]:
计算步骤：
v₁ = x       → v₁ = 2, ẋ = 1 (导数)
v₂ = y       → v₂ = 3, ẏ = 0 (对x求导时y视为常数)
v₃ = v₁ * v₂ → v₃ = 6, ˙v₃ = ẋ*v₂ + v₁*ẏ = 1*3 + 2*0 = 3
v₄ = sin(v₁) → v₄ = sin(2), ˙v₄ = cos(v₁)*ẋ = cos(2)*1
v₅ = v₃ + v₄ → v₅ = 6 + sin(2), ˙v₅ = ˙v₃ + ˙v₄ = 3 + cos(2)

![前向传播.png](../pic/前向模式.png "前向模式，类似这个图") 

### 反向模式
* 原理：先正向计算函数值，再反向从输出到输入逐层计算导数
* 适用场景：输出维度低，输入维度高
* 过程：
  * 正向传播：计算所有的中间变量
  * 反向传播：计算梯度，从输出到输入逐层累计
* 示例
  还是以上面函数为例，在点(x = 2, y = 3) 处计算梯度 $\nabla f = [\frac{\partial f}{\partial x}, \frac{\partial f}{\partial y}]$

In [ ]:
正向计算：
v₁ = x = 2
v₂ = y = 3
v₃ = v₁ * v₂ = 6
v₄ = sin(v₁) ≈ 0.909
v₅ = v₃ + v₄ ≈ 6.909

反向传播（从v₅开始）：
∂v₅/∂v₅ = 1
∂v₅/∂v₃ = 1 → ∂f/∂v₃ = 1
∂v₅/∂v₄ = 1 → ∂f/∂v₄ = 1

∂v₃/∂v₁ = v₂ = 3 → ∂f/∂v₁ += ∂f/∂v₃ * 3 = 1*3 = 3
∂v₃/∂v₂ = v₁ = 2 → ∂f/∂v₂ += ∂f/∂v₃ * 2 = 1*2 = 2

∂v₄/∂v₁ = cos(v₁) ≈ cos(2) ≈ -0.416 → ∂f/∂v₁ += ∂f/∂v₄ * (-0.416) = 1*(-0.416) ≈ -0.416

最终梯度：
∂f/∂x = ∂f/∂v₁ ≈ 3 - 0.416 ≈ 2.584
∂f/∂y = ∂f/∂v₂ = 2

### 雅可比矩阵
雅可比矩阵是多变量函数的一阶偏导数矩阵，对于函数 $\mathbf{f}: \mathbb{R}^n \rightarrow \mathbb{R}^m $，其雅可比矩阵 $\mathbf{J} \in \mathbb{R}^{m \times n}$ 可以表示为：
$$\mathbf{J}_{ij} = \frac{\partial f_i}{\partial x_j}$$

对于前向模式，使用雅可比向量积可以表示在输入方向向量 $\mathbf{V} \in \mathbb{R}^n$ 上的导数，表达式为
$$\mathbf{J_V} = \mathbf{J} \cdot \mathbf{V}  \in \mathbb{R}^m$$
工程中实际按照上面步骤实现逐层实现前向模式自动微分计算，不会显示构建雅可比矩阵。

对于后向模式，使用向量雅可比积表示输出对输入的梯度，后向模式的向量是输出梯度方向。

## 自动微分实现方式
1. 表达式或图(LIB)：封装基本的表达式及其微分表达式作为库函数。这样是需要先定义计算图，对于运算不会真正的执行，只有在调用时才会真正执行。例如TensorFlow 1.x中
2. 操作符重载(OO)：使用操作符重载基本运算表达式，这样在代码中可以直接使用原始运算符
3. 源码转换法(AST)：昇思的实现思路，将所有的表达式封装为函数

In [ ]:
# 1.表达式
x1 = xxx
x2 = xxx
t1 = ADlog(x1)
t2 = ADSin(x2)
t3 = ADMul(x1, x2)
t4 = ADAdd(t1, t3)
t5 = ADSub(t4, t2)

# /图法
# 预先定义计算图（类似写食谱）
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
c = a + b  # 这里只是定义了一个计算节点，不会真正执行加法！

# 运行时传入数据
with tf.Session() as sess:
    result = sess.run(c, feed_dict={a: 1, b: 2})  # 实际执行计算

In [ ]:
# 2.操作符重载，以 PyTorch 为例
a = torch.tensor(1.0, requires_grad=True)
b = torch.tensor(2.0, requires_grad=True)
c = a + b  # 这里直接执行加法，同时记录计算图！
c.backward()  # 自动求导

## 参考资料
[ZOMI酱AI系统](https://github.com/chenzomi12/AISystem)